# 3.1.2

In [28]:
import numpy as np
from itertools import product

N = 2
n = 6

C_raw = [0.1*N*i*j for i, j in product(range(1, n + 1), range(1, n + 1))]
A_raw = np.array([(125/(4 + c*0.25)**6) for c in C_raw])
A = A_raw.reshape((n, n))

### 1. Задать систему и найти решение

In [10]:
b = np.array([np.float64(N)]*n)
print('A:', A, '\n', 'b:', b)

A: [[0.02832565 0.02631521 0.02446929 0.02277269 0.02121175 0.01977422]
 [0.02631521 0.02277269 0.01977422 0.01722638 0.01505341 0.01319359]
 [0.02446929 0.01977422 0.01609719 0.01319359 0.01088299 0.00903095]
 [0.02277269 0.01722638 0.01319359 0.01022028 0.008      0.00632252]
 [0.02121175 0.01505341 0.01088299 0.008      0.00596972 0.00451579]
 [0.01977422 0.01319359 0.00903095 0.00632252 0.00451579 0.00328354]] 
 b: [2. 2. 2. 2. 2. 2.]


In [11]:
x = np.linalg.solve(A, b)
print('Solution:', x)

Solution: [  634.55255727 -2489.26761738  5081.29044241 -5708.30702852
  3353.92805949  -806.77186167]


### 2. Найти  число обусловленности матрицы А

In [21]:
cond = np.linalg.cond(A, p=np.inf)
print('Condition number :', cond)

Condition number : 16644654.262481416


### 3. Найти решения $Ax^i = b^i$

In [15]:
eps = 0.001

bs = np.repeat(b, n).reshape((n, n))
diag = np.diag([eps]*n)
bs = bs + diag


sols = []
for b in bs:
    sols.append(np.linalg.solve(A, b))

### 4. Найти относительные погрешности решений $x^i$

In [17]:
norms = list(map(lambda x_i: max(abs(x - x_i))/max(abs(x)), sols))
norms = np.array(norms, dtype=np.float64)
print(norms)

[0.22822803 1.70611743 4.9809452  7.11991749 4.99527222 1.37891053]


### 5. Найти самый весомый компонент вектора $b$

In [20]:
comp = np.argsort(norms)[-1]
assert max(np.abs(norms)) == norms[comp]
print('Max component of vector b:', comp)

Max component of vector b: 3


### 6. Результаты

In [22]:
estimate = cond * (max(np.abs(b - bs[comp]))/max(abs(b)))
print('Theoretical:', estimate, '\n', 'Practical:', norms[comp])

Theoretical: 8318.168047216184 
 Practical: 7.119917487115834


Вывод: практическая оценка на много меньше теоретической. Так получилось из-за большого числа обусловленности, которое участвует в формуле для теоретической оценки. Но оценка на практике не превосходит теоретического супремума, поэтому решение, полученное для зашумленных данных (которое соответствует теоретической оценке), является адекватным.

# 3.4

In [26]:
def getUpper(matrix, n):
    for i, j in product(range(n), range(n)):
        matrix[i, j] = 0 if i > j else matrix[i, j]
    return matrix

def getLower(matrix, n):
    for i, j in product(range(n), range(n)):
        if i < j:
            matrix[i, j] = 0
        elif i == j:
            matrix[i, j] = 1
        else:
            matrix[i, j] = matrix[i, j]
    return matrix

In [30]:
from scipy.linalg import lu_factor

LU, p = lu_factor(A)

lower = getLower(LU.copy(), n)
upper = getUpper(LU.copy(), n)

# 3.9.1

In [31]:
def cholesky(A):
    L = np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(i + 1):
            if i != j:
                L[i,j] = 1 / L[j,j] * (A[i,j] - np.dot(L[i], L[j]))
            else:
                L[i,i] = np.sqrt(A[i,i] - np.dot(L[i], L[i]))
    return L            

In [33]:
N = 1
n = 40
m = 10

b = np.array([n * i + m for i in range(n)])
A = np.array([
    [(i + j)/(m + n) if i != j else (n + m**2 + j/m + i/n) for j in range(n)]
                                for i in range(n)])

In [34]:
L = cholesky(A)
y = np.linalg.solve(L, b)
x = np.linalg.solve(L.T,y)
A_inv = np.dot(np.linalg.inv(L.T),np.linalg.inv(L))

cond = np.linalg.norm(A, ord = np.inf)*np.linalg.norm(A_inv, ord = np.inf)
cond

1.6592352728067103